In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install /content/drive/MyDrive/

Processing ./drive/MyDrive
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 125 kB 5.1 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 3.0 MB 50.3 MB/s 
     |████████████████████████████████| 12.8 MB 31.1 MB/s 
     |████████████████████████████████| 5.2 MB 20.0 MB/s 
     |████████████████████████████████| 212 kB 79.1 MB/s 
     |████████████████████████████████| 99 kB 9.8 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
  Creat

In [5]:
from fast_bert.data_abs import BertAbsDataBunch
from fast_bert.learner_abs import BertAbsLearner
from box import Box
import logging
import torch
from pathlib import Path
from transformers import BertTokenizer

In [10]:
from tokenizers.implementations.bert_wordpiece import BertWordPieceTokenizer
from tokenizers.implementations.sentencepiece_bpe import SentencePieceBPETokenizer
from tokenizers.implementations.byte_level_bpe import ByteLevelBPETokenizer
# from tokenizers.implementations import BPETokenizer

In [11]:
# 需要准备数据，才能跑以下代码

In [ ]:
path = '/content/drive/MyDrive'
import os

PATH = Path(os.path.join(path, "sample_data/summarisation/"))
DATA_PATH = PATH/'data'
MODEL_PATH = PATH/'model'

In [12]:
logger = logging.getLogger()

In [13]:
args = Box({
    "max_seq_length": 512,
    "batch_size": 8,
    "learning_rate": 5e-3,
    "num_train_epochs": 6,
    "fp16": True,
    "model_name": 'bertabs-finetuned-cnndm',
    "model_type": 'bert'
})

In [ ]:
device = torch.device('cuda') if torch.cuda.device_count() else torch.device('cpu')
if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False

In [ ]:
tokenizer = BertWordPieceTokenizer(str(MODEL_PATH/'vocab.txt'), lowercase=True)

In [ ]:
databunch = BertAbsDataBunch(data_dir=DATA_PATH, tokenizer=tokenizer, device=device)
databunch_old = BertAbsDataBunch(data_dir=DATA_PATH, tokenizer=args.model_name, device=device)

In [ ]:
databunch_with_data = BertAbsDataBunch(data_dir=DATA_PATH, tokenizer=args.model_name, device=device)

In [ ]:
databunch_with_data_new_tokeniser = BertAbsDataBunch(data_dir=DATA_PATH, tokenizer=tokenizer, device=device)

In [ ]:
learner = BertAbsLearner.from_pretrained_model(databunch, MODEL_PATH, device, logger=logger)

In [ ]:
texts = databunch_with_data.test_dl.dataset[0][1]
texts

In [ ]:
%%timeit
learner.predict_batch(texts)

In [ ]:
learner_old = BertAbsLearner.from_pretrained_model(databunch_old, MODEL_PATH, device, logger=logger)

In [ ]:
%%timeit
learner_old.predict_batch(texts)